### xlsx 파일에서 뉴스 링크 가져오기
- 경향신문

In [27]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


file = pd.read_excel("NewsResult_20220216-20220516.xlsx")[["일자","언론사","기고자","제목","키워드","URL"]]
# URL 컬럼 존재

In [28]:
file.head(3)

,일자,언론사,기고자,제목,키워드,URL
0,20220516,경향신문,윤은용 기자 plaimstone@kyunghyang.com,지소연 ‘아름다운 피날레’,"지소연,피날레,잉글랜드,FA,맨시티,제압,첼시,2관왕,동행,마침표,찍어,에이스,한국...",https://www.khan.co.kr/sports/football/article...
1,20220516,경향신문,윤은용 기자 plaimstone@kyunghyang.com,"‘우리’와 4년 더 위성우 감독, 14시즌 동행 계약","감독,위성우,계약,시즌,동행,여자,프로,농구,역대,최고,명장,위성우,아산,우리은행,...",https://www.khan.co.kr/sports/basketball-volle...
2,20220516,경향신문,황민국 기자 stylelomo@kyunghyang.com,"한국, 중국이 포기한 ‘아시안컵’ 유치 재도전 나선다","한국,중국,포기,아시안컵,유치,중국,확산,코로나,급속,개최,반납,축구협회,실무회의,...",https://www.khan.co.kr/sports/football/article...


### 신문사 결합한 크롤링 코드

In [23]:
import urllib.request
from urllib.parse import quote
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import openpyxl
from openpyxl import Workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows

In [34]:
soup_dict = {"동아일보":"div.article_txt",
            "중앙일보":"div.article_body",
            "한겨례":"div.text"}

# 경향신문만 다르게


def Crawl_news(category, URL):
    
    response = requests.get(URL)
    html = response.text
    
    if category == "경향신문":
        req = Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
        source_code_from_url = urlopen(req).read()

        soup = BeautifulSoup(source_code_from_url,"html.parser")
        content = soup.select_one(f"div.art_body").text
        
    else:
        soup = BeautifulSoup(html,"html.parser")

        tag = soup_dict[category] # 각 신문사별 태그명

        content = soup.select_one(tag).text

    return content

In [ ]:
import time
from tqdm import tqdm

content_lst = []

for index,row in tqdm(file.iterrows(), total = file.shape[0]):
    content = Crawl_news(row["언론사"], row["URL"])
    content_lst.append(content)

# 크롤링 관련 글

- 링크 : https://playthegame00.tistory.com/59
- meta content 태그에 있는 텍스트는 전문이 아님

In [10]:
from bs4 import BeautifulSoup
import requests
import openpyxl
from openpyxl import Workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows


wb = Workbook()
sheet = wb.active

In [17]:
url = "https://www.donga.com/news/article/all/20220515/113400207/2"
response = requests.get("https://www.donga.com/news/article/all/20220515/113400207/2")

# text 메소드로 태그 내 텍스트만 추출
html = response.text

In [18]:
soup = BeautifulSoup(html,"html.parser")
print(soup.head.find("meta",{"name":"description"}))

<meta content="박지현 더불어민주당 공동비상대책위원장이 이준석 국민의힘 대표의 성 상납 의혹을 ‘개인 사생활 문제’라며 옹호한 권성동 국민의힘 원내대표를 향해 “성상납은 사생활이 아니라 범죄 행…" name="description"/>


- 링크 : https://velog.io/@macang15/%EB%84%A4%EC%9D%B4%EB%B2%84-%EB%89%B4%EC%8A%A4-%EA%B8%B0%EC%82%AC-%EC%A0%9C%EB%AA%A9%EB%82%A0%EC%A7%9C%EB%B3%B8%EB%AC%B8%EB%8C%93%EA%B8%80-%ED%81%AC%EB%A1%A4%EB%A7%81-%ED%95%98%EA%B8%B0-%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EA%B8%B0%EA%B0%84-%EC%84%A4%EC%A0%95-%EC%96%B8%EB%A1%A0%EC%82%AC-%EC%84%A4%EC%A0%95
- 동아일보 크롤링 성공

In [19]:
soup = BeautifulSoup(html,"html.parser")

# 뉴스기사 전문 크롤링
content = soup.select_one("div.article_txt").text
title = soup.head.find("meta",{"property":"og:title"}).get("content")
date = soup.head.find("meta",{"property":"dd:published_time"}).get("content")

# 뉴스기사 제목, 날짜

In [20]:
wb = Workbook()
sheet = wb.active
sheet.append(['발행일','기사제목',"URL",'내용'])
sheet.append([date, title,url,content])

In [21]:
wb.save(filename="1월.csv")

In [22]:
content

'\n박지현 더불어민주당 공동비상대책위원장이 이준석 국민의힘 대표의 성 상납 의혹을 ‘개인 사생활 문제’라며 옹호한 권성동 국민의힘 원내대표를 향해 “성상납은 사생활이 아니라 범죄 행위”라며 “수준이하”라고 직격했다. 박 위원장은 14일 오후 이광재 강원지사 후보 선거사무소 개소식에 참석해 인사말을 통해 “제가 민주당을 비판하려거든 이 대표 성 상납부터 징계하라고 했더니 (권 원내대표가) ‘성상납을 받는 것은 사생활’이라고 한다”며 이같이 말했다.그러면서 “이런 수준 이하 선수를 강원도에서 완전히 몰아내기 위해서라도 우리는 반드시 이겨야 한다”고 강조했다. 또 “이 사람(권성동 원내대표)이 강원도 사람이라는 것이 솔직히 조금 창피한 마음”이라고 비판했다. \n\n 앞서 권 원내대표는 이 대표의 성상납 의혹 징계 절차에 대해 “개인의 사생활에 관한 문제라 진행상황은 전혀 모르고 있고, 사생활에 관한 문제를 파악하는 것도 적절치 않다고 생각한다”고 말한 바 있다.박 위원장은 김진태 국민의힘 강원도지사 후보의 ‘5.18 망언’도 언급하면서 “우리 이광재 후보를 어찌 보고 국민의힘에서 김진태 후보를 내세웠나 싶다”며 “(김 후보는) 광주 북한군 침투설을 끊임없이 제기했던 사람”이라고 지적했다. 이어 “김진태 후보께서는 이미 사과하셨다고 하지만 공천이라는 절체절명의 이해관계 앞에서 한 사과는 진정한 사과가 아니다”라며 “(김 후보가) 후보직을 사퇴하는 것이 진정한 사과 아니겠느냐”고 덧붙였다. 또 “이분은 후보직을 사퇴하는 것이 진정한 사과 아니냐. 정치인의 역사 왜곡에 대한 책임은 도지사 자리보다 훨씬 무겁다는 것을 꼭 확인해주길 바란다”고 강조했다. 이날 개소식에 참석한 박홍근 민주당 원내대표 역시 김 후보를 향해 “숱한 허위사실을 퍼뜨리고 망언을 내뱉어 온 막말꾼 후보는 안 된다”며 “공약이 아무리 좋은들 지켜지기나 하겠느냐”고 지적했다. 송치훈 동아닷컴 기자 sch53@donga.com \n\n\n\n송치훈 기자\n동아닷컴 온라인뉴스팀\nsch53@donga.c

- 중앙일보 크롤링 성공
- 링크 : https://www.joongang.co.kr/article/25071233

In [75]:
url = "https://www.joongang.co.kr/article/25071233"
response = requests.get(f"https://www.joongang.co.kr/article/25071233")
# text 메소드로 태그 내 텍스트만 추출
html = response.text

soup = BeautifulSoup(html,"html.parser")
content = soup.select_one(f"div.article_body").text

content

"\n윤석열 정부의 초대 국무총리 후보자가 4월 3일 지명 이후 42일째 국회의 벽을 넘지 못하고 있다. 윤 대통령이 10일 ‘1호 결재’로 한덕수 후보자 임명동의안에 서명했지만, 더불어민주당은 인준 부결을 만지작거리고 있다. 윤 대통령이 11일 박진 외교부, 이상민 행정안전부 등 청문 보고서가 채택 안된 장관들의 임명을 강행하자 야당 내 '한덕수 불가론'이 점차 커지는 모양새다.\n   국정 전반을 총괄하는 국무총리는 다른 장관과 달리 대통령이 임명을 강행할 수 없고, 국회 본회의 표결을 거쳐야 한다. 재적 의원(300명) 과반 출석, 출석 의원 과반 찬성이 필요하다. 168석을 보유한 민주당은 마음만 먹으면 국민의힘(109석)을 완력으로 누르고 ‘한덕수 카드’를 뒤집을 수 있다.\n한덕수 후보자는 노무현 정부 말기인 2007년 국회 본회의에서 참여 의원 270명 중 210명의 찬성표를 얻어 무난하게 총리로 임명됐다. 사진은 2005년 3월 15일 청와대에서 노무현 대통령이 한덕수 당시 재경부총리에게 임명장을 수여하는 모습. 중앙포토\n   한 후보자는 노무현 정부 말기인 2007년만 해도 270명 중 210명의 찬성표를 얻어 무난하게 총리로 임명됐다. 하지만 당시 국회는 한·미 자유무역협정(FTA) 비준을 앞둔 특수 상황이었다. 여야가 아니라 FTA 찬성파와 반대파로 국회가 갈렸다. 이 때문에 FTA에 찬성하는 한나라당 의원들도 대부분 한 후보자에게 찬성표를 던졌다.\n   지금 국면은 전혀 다르다. ‘검수완박’(검찰 수사권 완전 박탈) 법안 통과를 두고 거칠게 충돌한 여야가 6월 1일 지방선거를 앞두고 대치 국면을 이어가고 있다. 국민의힘 관계자는 12일 통화에서 “의석 차이가 너무 나기 때문에 역대 어느 총리 인준 때보다 불리한 상황”이라고 말했다.\n김대중 대통령과 이한동 국무총리가 2001년 3월 27일 오전 청와대에서 열린 국무회의장에 들어서고 있다. 중앙포토\n   2000년 6월 인사청문제도 도입 이후 첫 국회 표결을 거친 이는 김대중 정부의 

- 한겨례 신문 크롤링 시도
- 링크 : https://www.hani.co.kr/arti/politics/politics_general/1042845.html?_fr=mt1

In [84]:
url = "https://www.hani.co.kr/arti/politics/politics_general/1042845.html?_fr=mt1"
response = requests.get(f"https://www.hani.co.kr/arti/politics/politics_general/1042845.html?_fr=mt1")
# text 메소드로 태그 내 텍스트만 추출
html = response.text

soup = BeautifulSoup(html,"html.parser")
content = soup.select_one(f"div.text").text

content

'\n\n\n\n\n인천 계양을 국회의원 보궐선거에 나서는 이재명 더불어민주당 6·1 지방선거 총괄선거대책위원장이 12일 오전 인천시 남동구 더불어민주당 인천시당에서 열린 기자간담회에서 인사말을 하고 있다. 연합뉴스\n\n\n\n☞한겨레S 뉴스레터 구독하기 https://bit.ly/319DiiE\n\n우리나라에는 세가지 전국 선거가 있습니다. 대통령 선거, 국회의원 총선거, 전국 동시 지방선거입니다. 국회의원과 지방자치단체장 및 지방의원 임기는 4년입니다. 국회의원 총선과 지방선거는 2년마다 엇갈려 치르는 일정이 정착되어 있습니다.\n따라서 총선과 지방선거 사이에는 일정한 규칙이 있습니다. 이를테면 국회의원이 지방선거 단체장으로 출마하려면 국회의원을 2년 만에 그만두거나 국회의원을 하고 2년을 쉬었다가 나와야 합니다. 단체장이나 지방의원이 국회의원 총선에 출마하는 경우도 마찬가지입니다.\n그런데 우리나라는 대통령 선거가 훨씬 더 중요합니다. 대선 결과에 따라 여당과 야당이 결정되기 때문입니다. 총선과 지방선거 중간중간에 임기 5년 대통령 선거 일정이 끼어들면서 여러가지 복잡한 상황이 발생합니다. 대통령이 임기 도중 탄핵이라도 당하면 한층 더 복잡해집니다.\n올해가 바로 그런 경우입니다. 3월9일 대통령 선거를 치르고 3개월도 안 된 6월1일에 지방선거를 치러야 합니다. 일정이 이렇게 된 이유는 물론 박근혜 전 대통령 탄핵 때문입니다. 3개월 만에 전국선거를 두번이나 치르는 것은 자원 낭비입니다. 우리는 이런 정도 정치 일정을 여야가 타협해서 조정할 수 있는 정치적 역량을 갖지 못했습니다. 부끄러운 일입니다.\n\n\n\n\n\n\n\n\n\n 역대 세번째, 대선 직후 전국선거\n대통령 선거 직후에 곧이어 전국선거를 한 것은 1987년 이후 지금까지 두차례였습니다.\n첫째, 1987년 개헌 때문에 특별한 정치 환경이 만들어졌습니다.\n1987년 6월 항쟁 뒤 여야 합의로 개정된 헌법의 부칙에 따라 1987년 12월16일 대통령 선거를 하고 ‘4개월10일’ 뒤인

- 링크 : https://www.khan.co.kr/politics/north-korea/article/202205161434001
- 경향신문 크롤링
- HTTP 403 에러 수정 : https://jaimemin.tistory.com/1672

In [40]:
import urllib.request
from urllib.parse import quote
from urllib.request import Request, urlopen

URL = "https://www.khan.co.kr/sports/football/article/202205162247015"
req = Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
source_code_from_url = urlopen(req).read()

In [42]:
soup = BeautifulSoup(source_code_from_url,"html.parser")
content = soup.select_one(f"div.art_body").text

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [43]:
content

'\n\n\n잉글랜드 FA컵, 맨시티 제압첼시 2연패에 2관왕 힘 보태며‘8년 동행’ 화려한 마침표 찍어\n\n\n\n\n\n\n\n\n\n첼시 위민 지소연이 16일 영국 런던 웸블리 스타디움에서 열린 2021~2022 잉글랜드 여자 FA컵 결승전에서 맨체스터 시티를 3-2로 꺾고 우승한 뒤 우승 트로피를 들고 기뻐하고 있다. 런던 | EPA연합뉴스                            \n\n\n\n한국 여자축구 ‘에이스’ 지소연(31)이 첼시 위민의 잉글랜드축구협회(FA)컵 2연패와 시즌 2관왕에 힘을 보태며 8년 동행에 화려한 마침표를 찍었다. \n\n첼시 위민은 16일 영국 런던의 웸블리 스타디움에서 열린 2021~2022 잉글랜드 여자 FA컵 결승전에서 결승골을 포함해 멀티골을 터뜨린 샘 커의 활약을 앞세워 연장전 끝에 맨체스터 시티(이하 맨시티)를 3-2로 따돌리고 우승을 차지했다. \n\n이로써 첼시는 2020~2021시즌에 이어 FA컵 2연패를 달성하고, 통산 4번째 정상에 올랐다. 아울러 이번 시즌 잉글랜드 여자슈퍼리그(WSL)에 이어 2번째 트로피를 들어올렸다. \n\n이번 시즌을 끝으로 영국 생활을 마치고 한국으로 오기로 한 지소연은 이날 후반 24분 베서니 잉글랜드 대신 투입돼 여자 FA컵 사상 최다 관중인 4만9094명 앞에서 첼시 선수로서 마지막 경기를 치렀다. 후반 44분 헤일리 라소에게 2-2 동점골을 내줘 연장전으로 끌려간 첼시는 연장 전반 9분 간판 공격수 커의 득점포로 승기를 잡았다. 우승 확정을 눈앞에 둔 연장 후반 14분엔 엠마 헤이스 감독이 지소연을 마렌 미엘데로 교체해, 지소연에게 박수를 받으며 떠날 시간을 마련해줬다. \n \n\n \n2014년 첼시에 입단한 지소연은 줄곧 주축으로 활약하며 팀의 역사를 함께 썼다. 첼시에서 공식전 210경기에 출전해 68골을 터뜨렸는데, 출전 수와 득점 모두 첼시 위민 역대 3위에 해당한다. 첼시 유니폼을 입고 지소연은 WSL 6회, FA컵 4회, 리그컵 2회, 커뮤니티 